In [ ]:
import uproot
import matplotlib.pyplot as plt 
import pandas as pd 
import numpy as np 
import matplotlib as mpl

from util import *
import var
import cut
import data
import hist

import importlib

from pyanalib import panda_helpers

In [ ]:
importlib.reload(var)
importlib.reload(cut)
importlib.reload(data)
importlib.reload(hist)
importlib.reload(panda_helpers)

In [ ]:
mpl.rc('font', size=14)

dosave = False
savedir = "./plots-5-15/tunedsig_"

import warnings

warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

In [ ]:
filedir = "/icarus/data/users/gputnam/DMCP2023G/"

onbeamfile = filedir + "data/majority_prescaled_trk.df"
offbeamfile = filedir + "data/offbeam_unblind_trk.df"
mcfile = filedir + "mc/nu_trk_old.df"

mcfile = filedir + "mc/nu_fitwvfkstunegain_trk.df"

In [ ]:
mc = data.mc_dataset(mcfile, "trk")
mc.df

In [ ]:
# mchitdf = pd.read_hdf(mcfile, "hit")
# mchitdf

In [ ]:
onbeam = data.onbeam_dataset(onbeamfile, "trk")
onbeam.df

In [ ]:
offbeam = data.offbeam_dataset(offbeamfile, "trk")
offbeam.df

In [ ]:
importlib.reload(var)
importlib.reload(cut)
importlib.reload(data)
importlib.reload(hist)

In [ ]:
POTSTR = "$%.2f \\times 10^{18}$ POT" % (onbeam.POT/1e18)

In [ ]:
POTSTR

In [ ]:
@var.VAR
def first(v):
    nlevel = v.index.nlevels
    return v.groupby(level=list(range(nlevel-1))).first()

In [ ]:
bins = np.linspace(-1, 1, 21)

hd = hist.plotdata(plt, first@var.crlongtrkdiry, onbeam.slc, offbeam.slc, cut=first@cut.fiducial, bins=bins)
hmc = hist.plotmc(plt, first@var.crlongtrkdiry@var.slc, mc, cut=first@cut.fiducial@var.slc, 
                bins=bins, POT=onbeam.POT, fluxsyst=True, geniesyst=True, wgtf=first,
               categories=map(lambda f: first@f@var.slc, cut.mc_nu_cosmic_categories))

hist.makelegend(plt, hmc, hd, mc_labels=cut.mc_nu_cosmic_labels)
plt.tight_layout()

plt.title("Prescaled ICARUS Data")

plt.xlabel("CRLongTrkDirY")
plt.ylabel("Slices / %s" % POTSTR)
plt.tight_layout()
if dosave: plt.savefig(savedir + "crongtrkdiry.pdf")

In [ ]:
chi2p_bins = np.linspace(0, 400, 41)
chi2u_bins = np.linspace(0, 80, 41)

basecut = cut.fiducial@var.slc & (var.TRKVAR.len > 5)

@var.VAR
def ntrk(df):
    return df.groupby(level=[0,1,2]).size()

cuts = [
    basecut,
    basecut & (var.crlongtrkdiry@var.slc > -0.4) & (ntrk >= 3),
    basecut & (var.crlongtrkdiry@var.slc > -0.4) & (ntrk >= 3) & (var.TRKVAR.len > 100)
]

titles = [
    "Prescaled: Fiducial Tracks",
    "Prescaled: Fiducial, Cos. Rej. Tracks",
    "Prescaled: Fiducial, Cos. Rej., Long Tracks"
]

labels = ["fid", "fid_cosrej", "fid_cosrej_long"]

In [ ]:
1+1

In [ ]:
v = var.chi2p@var.trk

for i, c in enumerate(cuts):
    plt.figure(3*i)

    hmc = hist.plotmc(plt, v, mc, cut=c, bins=chi2p_bins, POT=onbeam.POT, fluxsyst=True, geniesyst=True,
                   hist_kw={"histtype": "step"})
    hd = hist.plotdata(plt, v, onbeam, offbeam, cut=c, bins=chi2p_bins)

    plt.xlabel("$\\chi^2_p$")
    plt.ylabel("Tracks / %s" % POTSTR)
    plt.title(titles[i])
    hist.makelegend(plt, hmc, hd)
    
    plt.tight_layout()
    if dosave: plt.savefig(savedir + "chi2p_%s.pdf" % labels[i])
    
    plt.figure(3*i+1)
    
    hmc = hist.plotmc(plt, v, mc, cut=c, bins=chi2p_bins, areanorm=True)
    hd = hist.plotdata(plt, v, onbeam, offbeam, cut=c, bins=chi2p_bins, areanorm=True)

    plt.xlabel("$\\chi^2_p$")
    plt.ylabel("Area Normalized")
    plt.title(titles[i])
    hist.makelegend(plt, hmc, hd)
    
    plt.tight_layout()
    if dosave: plt.savefig(savedir + "chi2p_areanorm_%s.pdf" % labels[i])
    
    plt.figure(3*i+2, figsize=(6.4*1.25, 4.8))
    hmc = hist.plotmc(plt, v, mc, cut=c, bins=chi2p_bins, POT=onbeam.POT, fluxsyst=True, geniesyst=True,
                    categories=map(lambda f: f@var.trk, cut.trk_mc_categories))
    
    hd = hist.plotdata(plt, v, onbeam, offbeam, cut=c, bins=chi2p_bins)
    hist.makelegend(plt, hmc, hd, mc_labels=cut.trk_mc_labels, split=True)
    
    plt.xlabel("$\\chi^2_p$")
    plt.ylabel("Tracks / %s" % POTSTR)
    plt.title(titles[i])
    
    plt.tight_layout()
    if dosave: plt.savefig(savedir + "chi2p_categories_%s.pdf" % labels[i])

In [ ]:
v = var.chi2u@var.trk

for i, c in enumerate(cuts):
    plt.figure(3*i)

    hmc = hist.plotmc(plt, v, mc, cut=c, bins=chi2u_bins, POT=onbeam.POT, fluxsyst=True, geniesyst=True,
                   hist_kw={"histtype": "step"})
    hd = hist.plotdata(plt, v, onbeam, offbeam, cut=c, bins=chi2u_bins)

    plt.xlabel("$\\chi^2_\\mu$")
    plt.ylabel("Tracks / %s" % POTSTR)
    plt.title(titles[i])
    hist.makelegend(plt, hmc, hd)

    
    plt.tight_layout()
    if dosave: plt.savefig(savedir + "chi2u_%s.pdf" % labels[i])
    
    plt.figure(3*i+1)
    
    hmc = hist.plotmc(plt, v, mc, cut=c, bins=chi2u_bins, areanorm=True)
    hd = hist.plotdata(plt, v, onbeam, offbeam, cut=c, bins=chi2u_bins, areanorm=True)

    plt.xlabel("$\\chi^2_\\mu$")
    plt.ylabel("Area Normalized")
    hist.makelegend(plt, hmc, hd)
    plt.title(titles[i])
    
    plt.tight_layout()
    if dosave: plt.savefig(savedir + "chi2u_areanorm_%s.pdf" % labels[i])
    
    plt.figure(3*i+2, figsize=(6.4*1.25, 4.8))
    hmc = hist.plotmc(plt, v, mc, cut=c, bins=chi2u_bins, POT=onbeam.POT, fluxsyst=True, geniesyst=True,
                    categories=map(lambda f: f@var.trk, cut.trk_mc_categories))
    
    hd = hist.plotdata(plt, v, onbeam, offbeam, cut=c, bins=chi2u_bins)
    hist.makelegend(plt, hmc, hd, mc_labels=cut.trk_mc_labels, split=True)

    plt.xlabel("$\\chi^2_\\mu$")
    plt.ylabel("Tracks / %s" % POTSTR)
    plt.title(titles[i])
    
    plt.tight_layout()
    if dosave: plt.savefig(savedir + "chi2u_categories_%s.pdf" % labels[i])

In [ ]:
bins = np.linspace(0, 1, 21)
plt.figure(figsize=(6.4*1.25, 4.8))
c = cut.fiducial@var.slc

v = var.trkscore@var.pfp

hmc = hist.plotmc(plt, v, mc, cut=c, bins=bins, POT=onbeam.POT, fluxsyst=True, geniesyst=True,
                 categories=map(lambda f: f@var.trk, cut.trk_mc_categories))
hd = hist.plotdata(plt, v, onbeam, offbeam, cut=c, bins=bins)

plt.ylabel("All Particles / %s" % POTSTR)
plt.xlabel("Pandora Track Score")
hist.makelegend(plt, hmc, hd, mc_labels=cut.trk_mc_labels, split=True)
plt.title("Prescaled ICARUS Data")

plt.tight_layout()
if dosave: plt.savefig(savedir + "particle_trkscore.pdf")

In [ ]:
bins = np.linspace(0, 1, 21)
plt.figure(figsize=(6.4*1.25, 4.8))
c = cut.fiducial@var.slc & (var.crlongtrkdiry@var.slc > -0.4)

v = var.trkscore@var.pfp

hmc = hist.plotmc(plt, v, mc, cut=c, bins=bins, POT=onbeam.POT, fluxsyst=True, geniesyst=True,
                 categories=map(lambda f: f@var.trk, cut.trk_mc_categories))
hd = hist.plotdata(plt, v, onbeam, offbeam, cut=c, bins=bins)

plt.ylabel("Particles / %s" % POTSTR)
plt.xlabel("Pandora Track Score")
hist.makelegend(plt, hmc, hd, mc_labels=cut.trk_mc_labels, split=True)
plt.title("Prescaled ICARUS Data")
plt.text(0.05, 0.9, "CRLongTrkDirY > -0.4", transform=plt.gca().transAxes, fontsize=12)

plt.tight_layout()
if dosave: plt.savefig(savedir + "cosmiccut_particle_trkscore.pdf")

In [ ]:
plt.figure(figsize=(6.4*1.25, 4.8))

bins = np.linspace(0, 1, 21)
c = cut.fiducial@var.slc & cut.is_proton@var.trk & cut.trkfiducial@var.trk

v = var.trkscore@var.pfp

hmc = hist.plotmc(plt, v, mc, cut=c, bins=bins, POT=onbeam.POT, fluxsyst=True, geniesyst=True,
                 categories=map(lambda f: f@var.trk, cut.trk_mc_categories))
hd = hist.plotdata(plt, v, onbeam, offbeam, cut=c, bins=bins)

plt.ylabel("Proton-Like Particles / %s" % POTSTR)
plt.xlabel("Pandora Track Score")
hist.makelegend(plt, hmc, hd, mc_labels=cut.trk_mc_labels, split=True)
plt.title("Prescaled ICARUS Data")
plt.text(0.05, 0.9, "$\\chi^2_p < 60$, $\\chi^2_\\mu > 40$", transform=plt.gca().transAxes, fontsize=12)

plt.tight_layout()
if dosave: plt.savefig(savedir + "proton_trkscore.pdf")

In [ ]:
plt.figure(figsize=(6.4*1.25, 4.8))

v = var.DF.pfp.trk.len
c = cut.fiducial@var.slc & cut.is_proton@var.trk & cut.trkfiducial@var.trk
length_bins = np.linspace(0, 100, 21)

hmc = hist.plotmc(plt, v, mc, cut=c, 
                bins=length_bins, POT=onbeam.POT, fluxsyst=True, geniesyst=True,
                categories=map(lambda f: f@var.trk, cut.trk_mc_categories))
hd = hist.plotdata(plt, v, onbeam, offbeam, cut=c, bins=length_bins)

hist.makelegend(plt, hmc, hd, mc_labels=cut.trk_mc_labels, split=True)
plt.ylabel("Proton-Like Tracks / %s" % POTSTR)
plt.xlabel("Track Length")

plt.title("Prescaled ICARUS Data")
plt.text(0.675, 0.25, "$\\chi^2_p < 60$, $\\chi^2_\\mu > 40$", transform=plt.gca().transAxes, fontsize=12)
plt.tight_layout()

if dosave: plt.savefig(savedir + "protonlike_track_length.pdf")

In [ ]:
# # Vary the calibration

# mchitdf["dqdx_e"] = mchitdf.dqdx*np.exp((mchitdf.t-850)*0.4/3e3)/0.01235

# # Try out systematic variations on chi2
# dedx_loA = var.dedxdf(dqdxname="dqdx_e", A=0.91)(mchitdf)
# dedx_hiA = var.dedxdf(dqdxname="dqdx_e", A=0.95)(mchitdf)

# dedx_loB = var.dedxdf(dqdxname="dqdx_e", B=0.212*0.95)(mchitdf)
# dedx_hiB = var.dedxdf(dqdxname="dqdx_e", B=0.212/0.95)(mchitdf)

In [ ]:
# Vary the truth

mchitdf["dqdx_e"] = mchitdf.dqdx*np.exp((mchitdf.t-850)*0.4/3e3)/0.01235

mchitdf["dqdx_e_loA"] = mchitdf.dqdx_e * var.scale_recombination(A=0.91)(mchitdf.truth.e / mchitdf.pitch)
mchitdf["dqdx_e_hiA"] = mchitdf.dqdx_e * var.scale_recombination(A=0.95)(mchitdf.truth.e / mchitdf.pitch)

# Try out systematic variations on recombination / gain
dedx_loA = var.dedxdf(dqdxname="dqdx_e_loA")(mchitdf)
dedx_hiA = var.dedxdf(dqdxname="dqdx_e_hiA")(mchitdf)

mchitdf["dqdx_e_loB"] = mchitdf.dqdx_e * var.scale_recombination(B=0.212*0.95)(mchitdf.truth.e / mchitdf.pitch)
mchitdf["dqdx_e_hiB"] = mchitdf.dqdx_e * var.scale_recombination(B=0.212*1.05)(mchitdf.truth.e / mchitdf.pitch)

# Try out systematic variations on chi2
dedx_loA = var.dedxdf(dqdxname="dqdx_e_loA")(mchitdf)
dedx_hiA = var.dedxdf(dqdxname="dqdx_e_hiA")(mchitdf)

dedx_loB = var.dedxdf(dqdxname="dqdx_e_loB")(mchitdf)
dedx_hiB = var.dedxdf(dqdxname="dqdx_e_hiB")(mchitdf)

In [ ]:
if "chi2u_loA" in mc.df: del mc.df["chi2u_loA"]
if "chi2p_loA" in mc.df: del mc.df["chi2p_loA"]
if "chi2u_hiA" in mc.df: del mc.df["chi2u_hiA"]
if "chi2p_hiA" in mc.df: del mc.df["chi2p_hiA"]

mc.df = panda_helpers.multicol_add(mc.df, var.hchi2u(dedx_loA).rename("chi2u_loA"))
mc.df = panda_helpers.multicol_add(mc.df, var.hchi2p(dedx_loA).rename("chi2p_loA"))
mc.df = panda_helpers.multicol_add(mc.df, var.hchi2u(dedx_hiA).rename("chi2u_hiA"))
mc.df = panda_helpers.multicol_add(mc.df, var.hchi2p(dedx_hiA).rename("chi2p_hiA"))

In [ ]:
if "chi2u_loB" in mc.df: del mc.df["chi2u_loB"]
if "chi2p_loB" in mc.df: del mc.df["chi2p_loB"]
if "chi2u_hiB" in mc.df: del mc.df["chi2u_hiB"]
if "chi2p_hiB" in mc.df: del mc.df["chi2p_hiB"]

mc.df = panda_helpers.multicol_add(mc.df, var.hchi2u(dedx_loB).rename("chi2u_loB"))
mc.df = panda_helpers.multicol_add(mc.df, var.hchi2p(dedx_loB).rename("chi2p_loB"))
mc.df = panda_helpers.multicol_add(mc.df, var.hchi2u(dedx_hiB).rename("chi2u_hiB"))
mc.df = panda_helpers.multicol_add(mc.df, var.hchi2p(dedx_hiB).rename("chi2p_hiB"))

In [ ]:
v = var.chi2u@var.trk
c = basecut & (var.crlongtrkdiry@var.slc > -0.4) & (ntrk >= 3) & (v > 0)

_, _, h1 = hist.plotmc(plt, v, mc, cut=c, bins=chi2u_bins, areanorm=True, 
                hist_kw={"histtype": "step"})
_, _, h2 = hist.plotmc(plt, var.DF.chi2u_loA, mc, cut=c, bins=chi2u_bins, areanorm=True, 
                hist_kw={"histtype": "step"})
_, _, h3 = hist.plotmc(plt, var.DF.chi2u_hiA, mc, cut=c, bins=chi2u_bins, areanorm=True, 
                hist_kw={"histtype": "step"})

_ = hist.plotdata(plt, v, onbeam, offbeam, cut=c, bins=chi2u_bins, areanorm=True)

plt.legend(handles = [h1[0], h2[0], h3[0]], labels=["Nominal", "Low $\\alpha$", "High $\\alpha$"])
plt.xlabel("$\\chi^2_\\mu$")
plt.ylabel("Area Normalized")

if dosave: plt.savefig(savedir + "areanorm_cosmicrej_chi2u_varyA.pdf")

In [ ]:
v = var.chi2u@var.trk
c = basecut & (var.crlongtrkdiry@var.slc > -0.4) & (ntrk >= 3) & (v > 0)

_, _, h1 = hist.plotmc(plt, v, mc, cut=c, bins=chi2u_bins, areanorm=True, 
                hist_kw={"histtype": "step"})
_, _, h2 = hist.plotmc(plt, var.DF.chi2u_loB, mc, cut=c, bins=chi2u_bins, areanorm=True, 
                hist_kw={"histtype": "step"})
_, _, h3 = hist.plotmc(plt, var.DF.chi2u_hiB, mc, cut=c, bins=chi2u_bins, areanorm=True, 
                hist_kw={"histtype": "step"})

_ = hist.plotdata(plt, v, onbeam, offbeam, cut=c, bins=chi2u_bins, areanorm=True)

plt.legend(handles = [h1[0], h2[0], h3[0]], labels=["Nominal", "Low $\\beta$", "High $\\beta$"])
plt.xlabel("$\\chi^2_\\mu$")
plt.ylabel("Area Normalized")

if dosave: plt.savefig(savedir + "areanorm_cosmicrej_chi2u_varyB.pdf")

In [ ]:
bins = np.linspace(0, 1, 21)
c = cut.fiducial@var.slc & cut.is_proton@var.trk

v = var.trkscore@var.pfp

_ = hist.plotmc(plt, v, mc, cut=c, bins=bins, areanorm=True)
_ = hist.plotdata(plt, v, onbeam, offbeam, cut=c, bins=bins, areanorm=True)

In [ ]:
bins = np.linspace(0, 1, 21)
c = cut.fiducial@var.slc & cut.is_proton@var.trk

v = var.chgendfrac@var.pfp

_ = hist.plotmc(plt, v, mc, cut=c, bins=bins, POT=onbeam.POT, fluxsyst=True)
_ = hist.plotdata(plt, v, onbeam, offbeam, cut=c, bins=bins)

In [ ]:
bins = np.linspace(0, 1, 21)
c = cut.fiducial@var.slc & cut.is_proton@var.trk

v = var.chgfracspread@var.pfp

_ = hist.plotmc(plt, v, mc, cut=c, bins=bins, POT=onbeam.POT)
_ = hist.plotdata(plt, v, onbeam, offbeam, cut=c, bins=bins)

In [ ]:
bins = np.linspace(0, 0.05, 21)
c = cut.fiducial@var.slc & cut.is_proton@var.trk

v = var.linfitdiff@var.pfp

_ = hist.plotmc(plt, v, mc, cut=c, bins=bins, POT=onbeam.POT)
_ = hist.plotdata(plt, v, onbeam, offbeam, cut=c, bins=bins)